In [ ]:
### Import libraries
from netCDF4 import Dataset
import xarray as xr
import numpy as np
import cartopy.crs as crs
import matplotlib
from matplotlib.cm import get_cmap
import matplotlib.pyplot as plt
from wrf import getvar, interplevel, smooth2d
import warnings
from scipy.interpolate import griddata
from matplotlib.cm import get_cmap
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
warnings.filterwarnings('ignore')
import matplotlib.transforms as mtransforms
import matplotlib.ticker as mticker

CP = 1005.7
RD = 287.04
P0 = 1000.
TR = 300.
LV = 2.501e6
EPS = 1.


In [ ]:
### Parameters
nx = 400
ny = 250
nz = 44
ti = 0
lev = 850

### Read in the data
ctl = Dataset('/p/work1/lloveras/real/nov2018/30km_files/ctl/wrfout_d01_2018-11-13_12_00_00')
box = Dataset('/p/work1/lloveras/real/nov2018/30km_files/adj_cape_box/wrfout_d01_2018-11-13_12_00_00')
hole = Dataset('/p/work1/lloveras/real/nov2018/30km_files/adj_cape_hole/wrfout_d01_2018-11-13_12_00_00')

### Latitude and longitude
lats = np.asarray(getvar(ctl,'lat',timeidx=ti))
lons = np.asarray(getvar(ctl,'lon',timeidx=ti))
lons[lons > 0] -= 360

### Pressure
p_ctl = np.asarray(getvar(ctl,'pressure',timeidx=ti))
p_box = np.asarray(getvar(box,'pressure',timeidx=ti))
p_hole = np.asarray(getvar(hole,'pressure',timeidx=ti))

### Height
z_ctl = np.asarray(getvar(ctl,'z',timeidx=ti))
zl_ctl = np.asarray(interplevel(z_ctl, p_ctl, lev))

### Theta
t_ctl = np.asarray(getvar(ctl,'THM',timeidx=ti))
tl_ctl = np.asarray(interplevel(t_ctl, p_ctl, lev))

t_box = np.asarray(getvar(box,'THM',timeidx=ti))
tl_box = np.asarray(interplevel(t_box, p_box, lev))
dtl_box = tl_box - tl_ctl

t_hole = np.asarray(getvar(hole,'THM',timeidx=ti))
tl_hole = np.asarray(interplevel(t_hole, p_hole, lev))
dtl_hole = tl_hole - tl_ctl


In [ ]:
### Contour interval
zlvls = np.asarray([-9,-7,-5,-3,-1,1,3,5,7,9])*0.25

### Plot subdomain
min_lat = 15
max_lat = 60
min_lon = -125
max_lon = -75

buf = 25
zl_ctl[(lons < min_lon-buf) | (lons > max_lon+buf)] = np.NaN

### Cartopy projection
proj = crs.LambertConformal(central_longitude=-100,standard_parallels=(20,70))

fig, axd = plt.subplot_mosaic([['left','right'],['cbar','cbar']],
                              constrained_layout=True, figsize=(6.9,3.6), dpi=2000, 
                              gridspec_kw={'width_ratios':[1,1],'height_ratios':[1,0.05]},
                              per_subplot_kw={'left':{'projection':proj},
                                              'right':{'projection':proj}})

trans = mtransforms.ScaledTranslation(1/30, -1/30, fig.dpi_scale_trans)

##############
### TOP LEFT
#############

# Add the gridlines
gls = axd['left'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['left'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=False
gls.rotate_labels=False

cs1 = axd['left'].contour(lons, lats, smooth2d(zl_ctl,2)/10.,
                  levels=np.arange(0,1500,5),
                  colors='k', transform=crs.PlateCarree())
axd['left'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,5),fontsize=6,colors='k')
im1 = axd['left'].contourf(lons, lats, dtl_box, levels=zlvls,
            cmap = get_cmap('RdBu_r'), extend='both',
            transform=crs.PlateCarree())

min_lat1 = 17
max_lat1 = 32
min_lon1 = -96
max_lon1 = -81

axd['left'].plot([min_lon1, min_lon1, max_lon1, max_lon1, min_lon1], 
                    [min_lat1, max_lat1, max_lat1, min_lat1, min_lat1], 
         color='g', linewidth=1.5, marker='None',
         transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['left'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['left'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['left'].set_title('CAPE Box')
axd['left'].text(0.0, 1.0, '(a)',transform=axd['left'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### TOP RIGHT
##############

# Add the gridlines
gls = axd['right'].gridlines(draw_labels=False, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['right'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])

cs1 = axd['right'].contour(lons, lats, smooth2d(zl_ctl,2)/10.,
                  levels=np.arange(0,1500,5),
                  colors='k', transform=crs.PlateCarree())
axd['right'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,5),fontsize=6,colors='k')
im1 = axd['right'].contourf(lons, lats, dtl_hole, levels=zlvls,
            cmap = get_cmap('RdBu_r'), extend='both',
            transform=crs.PlateCarree())

axd['right'].plot([min_lon1, min_lon1, max_lon1, max_lon1, min_lon1], 
                    [min_lat1, max_lat1, max_lat1, min_lat1, min_lat1], 
         color='g', linewidth=1.5, marker='None',
         transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['right'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['right'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)


# Title
axd['right'].set_title('CAPE Hole')
axd['right'].text(0.0, 1.0, '(b)',transform=axd['right'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

### SET THE COLORBAR AND SHOW
cbar = fig.colorbar(im1, orientation='horizontal', cax=axd['cbar'])
plt.savefig('/p/work1/lloveras/real/nov2018/figs_nov2018/4km_cape_setup.pdf',bbox_inches='tight')
plt.show()
